## Pre requists

In [15]:
!pip install --upgrade google-cloud-bigquery google-cloud-bigquery-storage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 3.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 32.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [16]:
!pip list | grep google-cloud-bigquery

google-cloud-bigquery         3.17.2
google-cloud-bigquery-storage 2.24.0

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [23]:
!python3 -m venv env_name
!source env_name/bin/activate
!pip install google-cloud-bigquery google-cloud-bigquery-storage
from google.cloud import bigquery_storage_v1

In [7]:
!pip install db-dtypes


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install pandas-gbq

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pulp import *
import pandas as pd
import os, glob

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/fa/Downloads/google-cloud-sdk/som-nero-phi-jonc101-2116f9f7bc68.json' 
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-jonc101' 
%load_ext google.cloud.bigquery

from google.cloud import bigquery
client=bigquery.Client()

## Base Cohort

In [2]:
%%bigquery df_uc_all
WITH urine_cultures AS ( -- all urine culture to be extracted 
    SELECT DISTINCT
        op.anon_id, 
        op.pat_enc_csn_id_coded as encounterid,
        op.order_proc_id_coded, 
        op.order_time_jittered_utc, 
        op.ordering_mode
    FROM 
        `som-nero-phi-jonc101.shc_core_2023.order_proc` op
     INNER JOIN
        `som-nero-phi-jonc101.shc_core_2023.lab_result` lr
    ON
        op.order_proc_id_coded = lr.order_id_coded
    WHERE
        op.order_type LIKE "Microbiology%"
        AND op.description LIKE "%URINE%"
        AND EXTRACT(YEAR FROM op.order_time_jittered_utc)>=2015 
        AND EXTRACT(YEAR FROM op.order_time_jittered_utc)<=2021
),
adult_urine_cultures AS (
    SELECT DISTINCT
        uc.*
    FROM 
        urine_cultures uc
    INNER JOIN
        `som-nero-phi-jonc101.shc_core_2023.demographic` demo
    USING
        (anon_id)
    WHERE
        DATE_DIFF(CAST(uc.order_time_jittered_utc as DATE), demo.BIRTH_DATE_JITTERED, YEAR) >= 18
),


-- Must not have any urine culture orders (that go on to result in lab results) in prior two weeks
-- This finds urine cultures that do have other urine cultures orders in prior two weeks
order_in_prior_two_weeks AS (
    SELECT DISTINCT
         auc.order_proc_id_coded
    FROM 
        `som-nero-phi-jonc101.shc_core_2023.order_proc` op
     INNER JOIN
        `som-nero-phi-jonc101.shc_core_2023.lab_result` lr
    ON
        op.order_proc_id_coded = lr.order_id_coded
    INNER JOIN
        adult_urine_cultures auc 
    ON
        op.anon_id = auc.anon_id
    WHERE
        op.order_type LIKE "Microbiology%"
        AND op.description LIKE "%URINE%"
        AND auc.order_time_jittered_utc > op.order_time_jittered_utc
        AND TIMESTAMP_DIFF(auc.order_time_jittered_utc, op.order_time_jittered_utc, DAY) < 14
),
-- this is the inclution urine culture patients 
included_adult_urine_cultures AS (
    SELECT DISTINCT
        auc.*
    FROM 
        adult_urine_cultures auc
    WHERE 
        auc.order_proc_id_coded NOT IN (SELECT order_proc_id_coded  FROM order_in_prior_two_weeks)
),
-- finf prior uti order up to 1 year before 
prior_utis AS (
    SELECT 
        auc.*,
        op.order_proc_id_coded as PriorUTI_proc_code,
        TIMESTAMP_DIFF(auc.order_time_jittered_utc, op.order_time_jittered_utc, Day) as TSLU,
    
        ROW_NUMBER() OVER (
            PARTITION BY op.anon_id
            ORDER BY ABS(TIMESTAMP_DIFF(auc.order_time_jittered_utc, op.order_time_jittered_utc, SECOND))
        ) AS rn
    FROM 
        `som-nero-phi-jonc101.shc_core_2023.order_proc` op
    INNER JOIN
        `som-nero-phi-jonc101.shc_core_2023.lab_result` lr
    ON
        op.order_proc_id_coded = lr.order_id_coded
    Right JOIN
        included_adult_urine_cultures auc 
    ON
        op.anon_id = auc.anon_id
    WHERE
        op.order_type LIKE "Microbiology%"
        AND op.description LIKE "%URINE%"
        AND auc.order_time_jittered_utc > op.order_time_jittered_utc
        AND TIMESTAMP_DIFF(auc.order_time_jittered_utc, op.order_time_jittered_utc, DAY) < 365
),
-- all included utis with time since last uti measurment
included_adult_urine_cultures_TSLU AS(
select *
from prior_utis 
where rn=1
),
-- Flag to indicated if the culture was positive, and resulting organisms
orders_that_were_positive AS (
    SELECT DISTINCT
        auc.order_proc_id_coded, 
    1 as was_positive,
    organism
    FROM
        included_adult_urine_cultures_TSLU auc
    INNER JOIN  
        (SELECT DISTINCT order_proc_id_coded, organism
        FROM `som-nero-phi-jonc101.shc_core_2023.culture_sensitivity`) cs
    USING
        (order_proc_id_coded) 
),
included_adult_urin_results_TSLU AS (
select * from
   included_adult_urine_cultures_TSLU auc
    left join 
    orders_that_were_positive using(order_proc_id_coded)
),
-- check other posetive culture within7 days of urin culture (e.g useful for silver labeling)
orders_with_other_pos_cultures AS (
    SELECT DISTINCT
        auc.order_proc_id_coded,
        1 as had_other_pos_culture,
        cs.description other_pos_site,
        cs.organism other_organism
    FROM
        included_adult_urin_results_TSLU auc
    INNER JOIN  
        `som-nero-phi-jonc101.shc_core_2023.culture_sensitivity` cs
    USING
        (anon_id) 
    WHERE
        auc.order_proc_id_coded <> cs.order_proc_id_coded
    AND
        -- Some urine culture orders have two order ids where only one results, make sure we're not counting these.
        -- Likely not needed logic after forcing orders to appear in lab_results
        (auc.order_time_jittered_utc <> cs.order_time_jittered_utc OR cs.description NOT LIKE '%URINE%')
    AND
        TIMESTAMP_DIFF(cs.order_time_jittered_utc, auc.order_time_jittered_utc, Day)
    BETWEEN
        0 AND 7
),
--Temporary building block that has medications and organisms in long form
included_adult_urin_results_TSLU_other_cult AS (
SELECT DISTINCT
    anon_id, 
    encounterid, 
    order_proc_id_coded, 
    order_time_jittered_utc,
    ordering_mode, 
    TSLU,
    CASE WHEN was_positive = 1 THEN 1 ELSE 0 END was_positive,
    organism,
    CASE WHEN had_other_pos_culture = 1 THEN 1 ELSE 0 END had_other_pos_culture,
    other_pos_site,
    other_organism
FROM 
    included_adult_urin_results_TSLU
LEFT JOIN 
    orders_with_other_pos_cultures
USING 
    (order_proc_id_coded)
ORDER BY
    order_proc_id_coded
)
select * from included_adult_urin_results_TSLU_other_cult;

/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/job/query.py:1892: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [3]:
df_uc_all.head()

,anon_id,encounterid,order_proc_id_coded,order_time_jittered_utc,ordering_mode,TSLU,was_positive,organism,had_other_pos_culture,other_pos_site,other_organism
0,JC1615868,131062588605,452572943,2015-01-02 20:00:00+00:00,Outpatient,38,0,None,0,None,None
1,JC1170548,131062572931,452582716,2015-01-01 21:46:00+00:00,Inpatient,273,0,None,0,None,None
2,JC704609,131062567081,452698731,2015-01-03 06:25:00+00:00,Inpatient,273,1,KLEBSIELLA PNEUMONIAE,0,None,None
3,JC1290431,131062509696,452728816,2015-01-04 17:55:00+00:00,Inpatient,36,0,None,0,None,None
4,JC1139743,131062855111,452756800,2015-01-03 22:26:00+00:00,Inpatient,34,0,None,0,None,None


In [4]:
df_uc_all=df_uc_all[['anon_id','encounterid','order_proc_id_coded','order_time_jittered_utc','ordering_mode','TSLU',
                     'was_positive','organism','had_other_pos_culture','other_pos_site','other_organism']]


In [5]:
# save the base cohort in big query for future usage
def string_agg_cols(arr):
    final_set = []
    for a in arr:
        if a is None:
            continue
        groups = a.split(',')
        for g in groups:
            if g not in final_set:
                final_set.append(g)
    final_set.sort()
    if len(final_set) == 0:
        return None
    else:
        return ','.join([f for f in final_set])
    
groupby_cols = ['anon_id', 'encounterid','order_time_jittered_utc']
df_final = (df_uc_all
    .groupby(groupby_cols)
    .agg({'order_proc_id_coded' : 'first',
          'ordering_mode' : 'first',
          'TSLU':'min',
          'was_positive' : 'max',
          'organism' : string_agg_cols,
          'had_other_pos_culture' : 'max',
          'other_pos_site' : string_agg_cols,
          'other_organism' : string_agg_cols})
    .reset_index()
)
print(df_final.shape)
df_final.head()


(32807, 11)


,anon_id,encounterid,order_time_jittered_utc,order_proc_id_coded,ordering_mode,TSLU,was_positive,organism,had_other_pos_culture,other_pos_site,other_organism
0,JC1000093,131310963922,2021-04-22 19:07:00+00:00,726257648,Outpatient,115,0,None,0,None,None
1,JC1000169,131216209091,2017-01-12 04:40:00+00:00,516543570,Outpatient,114,1,KLEBSIELLA PNEUMONIAE,0,None,None
2,JC1000322,131260044384,2018-11-30 02:43:00+00:00,583871965,Outpatient,189,0,None,0,None,None
3,JC1000441,131211600088,2016-12-19 05:10:00+00:00,513233094,Inpatient,162,0,None,0,None,None
4,JC1000514,131213337504,2017-02-06 22:39:00+00:00,518193267,Inpatient,54,0,None,0,None,None


In [6]:
df_final[df_final.had_other_pos_culture==1].head()

,anon_id,encounterid,order_time_jittered_utc,order_proc_id_coded,ordering_mode,TSLU,was_positive,organism,had_other_pos_culture,other_pos_site,other_organism
46,JC1004727,131235921589,2017-06-12 19:06:00+00:00,529910914,Outpatient,29,0,None,1,LAB ONLY GROUP B WORKUP,STREPTOCOCCUS AGALACTIAE (GROUP B)
76,JC1008147,131236120108,2017-06-27 08:40:00+00:00,530391801,Inpatient,101,1,ESCHERICHIA COLI,1,"VIA PHLEBOTOMY,BLOOD CULT - FIRST SET,BLOOD C...","COAG NEGATIVE STAPHYLOCOCCUS,ESCHERICHIA COLI"
134,JC1016525,131259360262,2019-01-11 16:55:00+00:00,588021746,Inpatient,21,0,None,1,RESPIRATORY CULTURE AND GRAM STAIN,KLEBSIELLA PNEUMONIAE
139,JC1017117,131182327528,2016-05-03 20:08:00+00:00,490260218,Outpatient,20,0,None,1,URINE CULTURE,STREPTOCOCCUS AGALACTIAE (GROUP B)
147,JC1017745,131188659767,2016-05-12 21:19:00+00:00,496379391,Outpatient,23,0,None,1,URINE CULTURE,ESCHERICHIA COLI


In [7]:
df_final.to_gbq(
    destination_table='Fatemeh_db.uti_cohort',
    project_id='som-nero-phi-jonc101',
    if_exists='replace'
)

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_48193/3579165325.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_final.to_gbq(


## recived antimicrobial with last 48 hours 

In [11]:
%%bigquery Antimicrobial_df
WITH cohort AS (
    SELECT * FROM som-nero-phi-jonc101.Fatemeh_db.uti_cohort
),
Antimicrobial_reciving as(
select op.*,
mm.RXCUI 
from `som-nero-phi-jonc101.shc_core_2023.order_med` op
left join (select * from `som-nero-phi-jonc101.shc_core_2023.mapped_meds` mm where RXCUI!='0') mm
on mm.name=op.med_description
where  op.thera_class_name in ('ANTIVIRALS','ANTIBIOTICS','ANTIINFECTIVES','ANTIFUNGALS','ANTIPARASITICS') 
)
select c.*,
1 as antimicrobail_recived,
ar.order_start_time_jittered_utc as antimicobial_start_time,
ar.order_end_time_jittered_utc as antimicobial_end_time,
ar.order_inst_jittered_utc as antimicobial_order_time,
from cohort c 
left join Antimicrobial_reciving ar 
using (anon_id) 
WHERE
TIMESTAMP_DIFF(ar.order_inst_jittered_utc,c.order_time_jittered_utc,HOUR) BETWEEN -24 AND 24 



/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/job/query.py:1892: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [12]:
Antimicrobial_df.head()

,anon_id,encounterid,order_time_jittered_utc,order_proc_id_coded,ordering_mode,TSLU,was_positive,organism,had_other_pos_culture,other_pos_site,other_organism,antimicrobail_recived,antimicobial_start_time,antimicobial_end_time,antimicobial_order_time,antimicrobail_recived_1,antimicobial_start_time_1,antimicobial_end_time_1,antimicobial_order_time_1
0,JC1564315,131262057833,2019-01-07 02:45:00+00:00,589386496,Inpatient,14,0,None,0,None,None,1,2019-01-08 00:30:00+00:00,2019-01-08 01:21:00+00:00,2019-01-08 00:09:00+00:00,1,2019-01-08 00:30:00+00:00,2019-01-08 01:21:00+00:00,2019-01-08 00:09:00+00:00
1,JC1564315,131262057833,2019-01-07 02:45:00+00:00,589386496,Inpatient,14,0,None,0,None,None,1,2019-01-08 00:30:00+00:00,2019-01-08 01:21:00+00:00,2019-01-08 00:09:00+00:00,1,2019-01-07 03:09:00+00:00,2019-01-07 03:27:00+00:00,2019-01-07 03:08:00+00:00
2,JC1564315,131262057833,2019-01-07 02:45:00+00:00,589386496,Inpatient,14,0,None,0,None,None,1,2019-01-08 00:30:00+00:00,2019-01-08 01:21:00+00:00,2019-01-08 00:09:00+00:00,1,2019-01-08 03:00:00+00:00,2019-01-07 23:44:00+00:00,2019-01-07 10:40:00+00:00
3,JC1564315,131262057833,2019-01-07 02:45:00+00:00,589386496,Inpatient,14,0,None,0,None,None,1,2019-01-08 03:00:00+00:00,2019-01-07 23:44:00+00:00,2019-01-07 10:40:00+00:00,1,2019-01-08 00:30:00+00:00,2019-01-08 01:21:00+00:00,2019-01-08 00:09:00+00:00
4,JC1564315,131262057833,2019-01-07 02:45:00+00:00,589386496,Inpatient,14,0,None,0,None,None,1,2019-01-08 03:00:00+00:00,2019-01-07 23:44:00+00:00,2019-01-07 10:40:00+00:00,1,2019-01-07 03:09:00+00:00,2019-01-07 03:27:00+00:00,2019-01-07 03:08:00+00:00


In [13]:
Antimicrobial_df.to_gbq(
    destination_table='Fatemeh_db.uti_cohort',
    project_id='som-nero-phi-jonc101',
    if_exists='replace'
)

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_48193/3547218449.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  Antimicrobial_df.to_gbq(


## Medications recived in last 28 days  

In [14]:
%%bigquery prior_meds_df
WITH cohort AS (
    SELECT * FROM som-nero-phi-jonc101.Fatemeh_db.uti_cohort
),
med_reciving as(
select op.*,
mm.RXCUI,
from `som-nero-phi-jonc101.shc_core_2023.order_med` op
left join (select * from `som-nero-phi-jonc101.shc_core_2023.mapped_meds` mm where RXCUI!='0') mm
on mm.name=op.med_description
-- where  op.thera_class_name in ('ANTIVIRALS','ANTIBIOTICS','ANTIINFECTIVES','ANTIFUNGALS','ANTIPARASITICS') 
),
cohort_medication as (
select c.anon_id,
c.encounterid,
c.order_proc_id_coded,
ar.RXCUI as Prior_med_RXCUI,
ar.med_description as Prior_med,
ar.order_inst_jittered_utc as prior_med_order_time,
from cohort c 
left join med_reciving ar 
using (anon_id) 
WHERE
TIMESTAMP_DIFF(ar.order_inst_jittered_utc,c.order_time_jittered_utc,day) BETWEEN -28 AND 0 
),
cohort_agg_medication as (
select 
anon_id,
encounterid,
order_proc_id_coded,
STRING_AGG(DISTINCT Prior_med_RXCUI ORDER BY Prior_med_RXCUI) Prior_med_RXCUIs,
STRING_AGG(DISTINCT Prior_med ORDER BY Prior_med) Prior_meds,
from cohort_medication
group by anon_id,encounterid,order_proc_id_coded)
select c.* , cam.Prior_med_RXCUIs,Prior_meds from cohort c 
left join cohort_agg_medication cam using(anon_id,encounterid,order_proc_id_coded);

/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/job/query.py:1892: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [15]:
prior_meds_df.head()

,anon_id,encounterid,order_time_jittered_utc,order_proc_id_coded,ordering_mode,TSLU,was_positive,organism,had_other_pos_culture,other_pos_site,...,antimicrobail_recived,antimicobial_start_time,antimicobial_end_time,antimicobial_order_time,antimicrobail_recived_1,antimicobial_start_time_1,antimicobial_end_time_1,antimicobial_order_time_1,Prior_med_RXCUIs,Prior_meds
0,JC1564315,131262057833,2019-01-07 02:45:00+00:00,589386496,Inpatient,14,0,None,0,None,...,1,2019-01-08 00:30:00+00:00,2019-01-08 01:21:00+00:00,2019-01-08 00:09:00+00:00,1,2019-01-08 00:30:00+00:00,2019-01-08 01:21:00+00:00,2019-01-08 00:09:00+00:00,"1605101,161,17767,21949,25480,26225,2669,28889...","ACETAMINOPHEN 1,000 MG/100 ML (10 MG/ML) IV SO..."
1,JC1564315,131262057833,2019-01-07 02:45:00+00:00,589386496,Inpatient,14,0,None,0,None,...,1,2019-01-08 00:30:00+00:00,2019-01-08 01:21:00+00:00,2019-01-08 00:09:00+00:00,1,2019-01-07 03:09:00+00:00,2019-01-07 03:27:00+00:00,2019-01-07 03:08:00+00:00,"1605101,161,17767,21949,25480,26225,2669,28889...","ACETAMINOPHEN 1,000 MG/100 ML (10 MG/ML) IV SO..."
2,JC1564315,131262057833,2019-01-07 02:45:00+00:00,589386496,Inpatient,14,0,None,0,None,...,1,2019-01-08 00:30:00+00:00,2019-01-08 01:21:00+00:00,2019-01-08 00:09:00+00:00,1,2019-01-08 03:00:00+00:00,2019-01-07 23:44:00+00:00,2019-01-07 10:40:00+00:00,"1605101,161,17767,21949,25480,26225,2669,28889...","ACETAMINOPHEN 1,000 MG/100 ML (10 MG/ML) IV SO..."
3,JC1564315,131262057833,2019-01-07 02:45:00+00:00,589386496,Inpatient,14,0,None,0,None,...,1,2019-01-08 03:00:00+00:00,2019-01-07 23:44:00+00:00,2019-01-07 10:40:00+00:00,1,2019-01-08 00:30:00+00:00,2019-01-08 01:21:00+00:00,2019-01-08 00:09:00+00:00,"1605101,161,17767,21949,25480,26225,2669,28889...","ACETAMINOPHEN 1,000 MG/100 ML (10 MG/ML) IV SO..."
4,JC1564315,131262057833,2019-01-07 02:45:00+00:00,589386496,Inpatient,14,0,None,0,None,...,1,2019-01-08 03:00:00+00:00,2019-01-07 23:44:00+00:00,2019-01-07 10:40:00+00:00,1,2019-01-07 03:09:00+00:00,2019-01-07 03:27:00+00:00,2019-01-07 03:08:00+00:00,"1605101,161,17767,21949,25480,26225,2669,28889...","ACETAMINOPHEN 1,000 MG/100 ML (10 MG/ML) IV SO..."


In [16]:
prior_meds_df.to_gbq(
    destination_table='Fatemeh_db.uti_cohort',
    project_id='som-nero-phi-jonc101',
    if_exists='replace'
)

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_48193/1095018108.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  prior_meds_df.to_gbq(


## ICD codes

In [17]:
%%bigquery icd_cohort_df
WITH cohort AS (
    SELECT * FROM som-nero-phi-jonc101.Fatemeh_db.uti_cohort
),
prior_ICDs as(
    select c.anon_id,
   c.encounterid,
   c.order_proc_id_coded,
   STRING_AGG(DISTINCT dx_name ORDER BY dx_name) dx_name,
   STRING_AGG(DISTINCT icd9 ORDER BY icd9) icd9,
   STRING_AGG(DISTINCT icd10 ORDER BY icd10) icd10,

from cohort c left join `som-nero-phi-jonc101.shc_core_2023.diagnosis` icd 
on c.anon_id=icd.anon_id and c.encounterid=icd.pat_enc_csn_id_jittered
WHERE DATETIME_DIFF(c.order_time_jittered_utc, icd.start_date_jittered_utc, DAY) between 0 and 365
group by c.anon_id,c.encounterid,c.order_proc_id_coded 
)
select c.*,pi.dx_name,pi.icd9,pi.icd10 from cohort c left join prior_ICDs pi on pi.anon_id=c.anon_id and pi.encounterid=c.encounterid and c.order_proc_id_coded=pi.order_proc_id_coded;


/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/job/query.py:1892: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [18]:
icd_cohort_df.head()

,anon_id,encounterid,order_time_jittered_utc,order_proc_id_coded,ordering_mode,TSLU,was_positive,organism,had_other_pos_culture,other_pos_site,...,antimicobial_order_time,antimicrobail_recived_1,antimicobial_start_time_1,antimicobial_end_time_1,antimicobial_order_time_1,Prior_med_RXCUIs,Prior_meds,dx_name,icd9,icd10
0,JC512391,131257024073,2018-08-31 18:31:00+00:00,576030707,None,21,0,None,0,None,...,2018-08-31 05:10:00+00:00,1,2018-08-30 07:00:00+00:00,2018-09-05 06:59:00+00:00,2018-08-31 05:10:00+00:00,"2231,3827","CEPHALEXIN 500 MG PO CAPS,ENALAPRIL MALEATE 20...","Urinary tract infection,Urinary tract infectio...",599.0,N39.0
1,JC1153285,131284383033,2020-03-29 21:51:00+00:00,655705329,None,39,0,None,0,None,...,2020-03-29 19:38:00+00:00,1,NaT,2021-02-07 08:00:00+00:00,2020-03-29 19:38:00+00:00,"1307404,1331,219965,2231,4457,6960,723,7242,8591",AMOXICILLIN-POT CLAVULANATE 875-125 MG PO TABS...,"Urinary tract infection, site not specified",599.0,N39.0
2,JC535498,131252483437,2018-06-23 22:42:00+00:00,566265954,None,19,0,None,0,None,...,2018-06-23 22:55:00+00:00,1,2018-06-23 07:00:00+00:00,2018-07-04 06:59:00+00:00,2018-06-23 22:55:00+00:00,"1191,237159,82122","ASPIRIN 325 MG PO TABS,LEVOFLOXACIN 500 MG PO ...","Urinary tract infection,Urinary tract infectio...",599.0,N39.0
3,JC1754014,131301484336,2020-12-08 22:43:00+00:00,701476119,None,37,0,None,0,None,...,2020-12-08 00:54:00+00:00,1,2020-12-07 08:00:00+00:00,2021-04-06 07:00:00+00:00,2020-12-08 00:54:00+00:00,11124,VANCOMYCIN 125 MG PO CAPS,"Urinary tract infection, site not specified",599.0,N39.0
4,JC1597098,131077526920,2015-03-20 22:44:00+00:00,459691052,None,178,0,None,0,None,...,2015-03-20 16:03:00+00:00,1,2015-03-16 07:00:00+00:00,2015-05-05 07:00:00+00:00,2015-03-20 16:03:00+00:00,73645,"ALEVE PO,VALACYCLOVIR 500 MG PO TABS",Dysuria,788.1,R30.0


In [19]:
icd_cohort_df.to_gbq(
    destination_table='Fatemeh_db.uti_cohort',
    project_id='som-nero-phi-jonc101',
    if_exists='replace'
)

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_48193/3555587913.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  icd_cohort_df.to_gbq(


## Labs within last 14 days 

In [20]:
%%bigquery labs_df
WITH cohort AS (
    SELECT * FROM som-nero-phi-jonc101.Fatemeh_db.uti_cohort
)
  SELECT c.*,
    l.group_lab_name,
    l.lab_name, 
    CAST(l.ord_num_value AS FLOAT64) AS ord_num_value,
    l.order_time_jittered_utc as lab_order_time,
    l.taken_time_jittered_utc as lab_taken_time, 
    l.result_time_jittered_utc as lab_result_time, 
  CASE 
  WHEN upper(lab_name) IN ('WBC') THEN 'wbc'
  WHEN upper(lab_name) IN ('NEUTROPHIL, ABSOLUTE') THEN 'neutrophil'
  WHEN upper(lab_name) IN ('LYMPHOCYTE, ABSOLUTE') THEN 'lymphocyte'
  WHEN upper(lab_name) IN ('HEMOGLOBIN') THEN 'hgb'
  WHEN upper(lab_name) IN ('PLATELET COUNT') THEN 'plt'
  WHEN upper(lab_name) IN ('SODIUM, SER/PLAS') THEN 'sodium'
  WHEN upper(lab_name) IN ('BUN, SER/PLAS', 'UREA NITROGEN,SER/PLAS') THEN 'bun'
  WHEN upper(lab_name) IN ('CO2, SER/PLAS') THEN 'bicarb'
  WHEN upper(lab_name) IN ('CREATININE, SER/PLAS') THEN 'creat'
  WHEN upper(lab_name) IN ('AST (SGOT), SER/PLAS') THEN 'ast'
  WHEN upper(lab_name) IN ('ALT (SGPT), SER/PLAS') THEN 'alt'
  WHEN upper(lab_name) IN ('TOTAL BILIRUBIN, SER/PLAS', 'TOTAL BILIRUBIN') THEN 'bili'
  WHEN upper(lab_name) IN ('ALBUMIN, SER/PLAS') THEN 'album'
  WHEN upper(lab_name) IN ('GLUCOSE, SER/PLAS', 'GLUCOSE BY METER', 'GLUCOSE,ISTAT') THEN 'gluc'
  WHEN upper(lab_name) IN ('LACTATE, ISTAT', 'LACTATE, WHOLE BLD', 'LACTIC ACID', 'LACTATE, POC') THEN 'lactate'
  ELSE lab_name
  END as lab, 
  CASE
  WHEN REGEXP_CONTAINS(UPPER(reference_unit), r'K/UL|X10E3/UL|THOUSAND/UL|KUL|10X3/UL|10\\*3/UL') THEN 'K/UL'
  WHEN REGEXP_CONTAINS(UPPER(reference_unit), 'MEQ/L|MMOL/L|MMOLL|MM/L|MMOLE/L') THEN 'MMOL/L'
  WHEN REGEXP_CONTAINS(UPPER(reference_unit), 'G/DL') THEN 'G/DL'
  WHEN REGEXP_CONTAINS(UPPER(reference_unit), 'MG/L') THEN 'MG/L'
  WHEN REGEXP_CONTAINS(UPPER(reference_unit), 'MG/DL') THEN 'MG/DL'
  WHEN REGEXP_CONTAINS(UPPER(reference_unit), 'MM/HR|MM/H|^MM$') THEN 'MM/HR'
  WHEN REGEXP_CONTAINS(UPPER(reference_unit), 'IU/L|U/L|UNITS/L') THEN 'U/L'
  ELSE reference_unit
  END as unit,
  FROM
  `som-nero-phi-jonc101.shc_core_2023.lab_result` as l right join cohort c ON l.pat_enc_csn_id_coded = c.encounterid
  and l.anon_id=c.anon_id 
  WHERE
  (reference_unit IS NOT NULL) AND
  (TIMESTAMP_DIFF(l.result_time_jittered_utc, c.order_time_jittered_utc, day) between -14 and 0 )
  AND 
  ((upper(trim(group_lab_name)) IN ('CBC WITH DIFFERENTIAL', 'CBC', 'CBC WITH DIFF') AND upper(trim(lab_name)) IN ('WBC')) OR -- White blood cell count
   (upper(trim(group_lab_name)) IN ('CBC WITH DIFFERENTIAL') AND upper(trim(lab_name)) IN ('NEUTROPHIL, ABSOLUTE')) OR -- Absolute neutrophil count
   (upper(trim(group_lab_name)) IN ('CBC WITH DIFFERENTIAL') AND upper(trim(lab_name)) IN ('LYMPHOCYTE, ABSOLUTE')) OR -- Absolute lymphocyte count
   (upper(trim(group_lab_name)) IN ('CBC WITH DIFFERENTIAL', 'CBC', 'CBC WITH DIFF') AND upper(trim(lab_name)) IN ('HEMOGLOBIN')) OR -- Hemoglobin
   (upper(trim(group_lab_name)) IN ('CBC WITH DIFFERENTIAL', 'CBC', 'CBC WITH DIFF') AND upper(trim(lab_name)) IN ('PLATELET COUNT')) OR -- Platelets
   (upper(trim(group_lab_name)) IN ('METABOLIC PANEL, COMPREHENSIVE', 'METABOLIC PANEL, BASIC', 'CMP') AND upper(trim(lab_name)) IN ('SODIUM, SER/PLAS')) OR -- Sodium
   (upper(trim(group_lab_name)) IN ('METABOLIC PANEL, COMPREHENSIVE', 'METABOLIC PANEL, BASIC', 'CMP') AND upper(trim(lab_name)) IN ('CO2, SER/PLAS')) OR -- Serum bicarbonate
   (upper(trim(group_lab_name)) IN ('METABOLIC PANEL, COMPREHENSIVE', 'METABOLIC PANEL, BASIC', 'CMP') AND upper(trim(lab_name)) IN ('BUN, SER/PLAS', 'UREA NITROGEN,SER/PLAS')) OR -- BUN
   (upper(trim(group_lab_name)) IN ('METABOLIC PANEL, COMPREHENSIVE', 'METABOLIC PANEL, BASIC', 'CMP') AND upper(trim(lab_name)) IN ('CREATININE, SER/PLAS')) OR -- Creatinine    
   (upper(trim(group_lab_name)) IN ('METABOLIC PANEL, COMPREHENSIVE', 'CMP', 'HEPATIC FUNCTION PANEL A') AND upper(trim(lab_name)) IN ('AST (SGOT), SER/PLAS')) OR -- AST
   (upper(trim(group_lab_name)) IN ('METABOLIC PANEL, COMPREHENSIVE', 'CMP', 'HEPATIC FUNCTION PANEL A') AND upper(trim(lab_name)) IN ('ALT (SGPT), SER/PLAS')) OR -- ALT
   (upper(trim(group_lab_name)) IN ('METABOLIC PANEL, COMPREHENSIVE', 'CMP', 'HEPATIC FUNCTION PANEL A') AND upper(trim(lab_name)) IN ('TOTAL BILIRUBIN, SER/PLAS', 'TOTAL BILIRUBIN')) OR -- Total bilirubin
   (upper(trim(group_lab_name)) IN ('METABOLIC PANEL, COMPREHENSIVE', 'CMP', 'HEPATIC FUNCTION PANEL A') AND upper(trim(lab_name)) IN ('ALBUMIN, SER/PLAS')) OR -- Serum albumin
   (upper(trim(group_lab_name)) IN ('GLUCOSE BY METER', 'METABOLIC PANEL, COMPREHENSIVE', 'METABOLIC PANEL, BASIC', 'CMP', 'ISTAT CR, HCT/HB, CL , BUN, NA, K, GLUCOSE, ICA, TCO2') AND upper(trim(lab_name)) IN ('GLUCOSE, SER/PLAS', 'GLUCOSE BY METER', 'GLUCOSE,ISTAT')) OR -- Serum or fingerstick glucose
  (upper(trim(group_lab_name)) IN ('ISTAT ARTERIAL BLOOD GASES AND LACTATE', 'ISTAT VENOUS BLOOD GASES AND LACTATE', 'LACTIC ACID', 'LACTATE, WHOLE BLOOD', 'LACTIC ACID, AUTO-REPEAT IN 3HR IF 1ST LACTIC RESULT >2', 'ISTAT OTHERS BLOOD GASES AND LACTATE') AND upper(trim(lab_name)) IN ('LACTATE, ISTAT', 'LACTATE, WHOLE BLD', 'LACTIC ACID', 'LACTATE, POC'))) -- Serum, whole blood, or iSTAT lactate


/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/job/query.py:1892: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [23]:
labs_df.drop_duplicates(inplace=True)
labs_df

,anon_id,encounterid,order_time_jittered_utc,order_proc_id_coded,ordering_mode,TSLU,was_positive,organism,had_other_pos_culture,other_pos_site,...,icd9,icd10,group_lab_name,lab_name,ord_num_value,lab_order_time,lab_taken_time,lab_result_time,lab,unit
0,JC641233,131171297317,2016-01-30 00:27:00+00:00,483545477,Inpatient,42,1,ESCHERICHIA COLI,0,None,...,"038.9, 995.91, 599.0,250.70, 443.81,272.4,277....","A41.9,A41.9, N39.0,D51.9,E11.22,E11.51,E78.5,E...","Metabolic Panel, Comprehensive","Sodium, Ser/Plas",141.00,2016-01-29 20:47:00+00:00,2016-01-29 20:48:00+00:00,2016-01-29 21:34:00+00:00,sodium,MMOL/L
1,JC641233,131171297317,2016-01-30 00:27:00+00:00,483545477,Inpatient,42,1,ESCHERICHIA COLI,0,None,...,"038.9, 995.91, 599.0,250.70, 443.81,272.4,277....","A41.9,A41.9, N39.0,D51.9,E11.22,E11.51,E78.5,E...",iSTAT Venous Blood Gases And Lactate,"Lactate, ISTAT",2.39,2016-01-29 23:07:00+00:00,2016-01-29 22:54:00+00:00,2016-01-29 23:07:00+00:00,lactate,MMOL/L
2,JC641233,131171297317,2016-01-30 00:27:00+00:00,483545477,Inpatient,42,1,ESCHERICHIA COLI,0,None,...,"038.9, 995.91, 599.0,250.70, 443.81,272.4,277....","A41.9,A41.9, N39.0,D51.9,E11.22,E11.51,E78.5,E...",CBC,Platelet count,201.00,2016-01-30 06:00:00+00:00,2016-01-30 14:08:00+00:00,2016-01-30 14:30:00+00:00,plt,K/UL
3,JC641233,131171297317,2016-01-30 00:27:00+00:00,483545477,Inpatient,42,1,ESCHERICHIA COLI,0,None,...,"038.9, 995.91, 599.0,250.70, 443.81,272.4,277....","A41.9,A41.9, N39.0,D51.9,E11.22,E11.51,E78.5,E...",CBC,Platelet count,201.00,2016-01-30 06:00:00+00:00,2016-01-30 14:08:00+00:00,2016-01-30 14:30:00+00:00,plt,K/UL
4,JC641233,131171297317,2016-01-30 00:27:00+00:00,483545477,Inpatient,42,1,ESCHERICHIA COLI,0,None,...,"038.9, 995.91, 599.0,250.70, 443.81,272.4,277....","A41.9,A41.9, N39.0,D51.9,E11.22,E11.51,E78.5,E...",CBC,Platelet count,201.00,2016-01-30 06:00:00+00:00,2016-01-30 14:08:00+00:00,2016-01-30 14:30:00+00:00,plt,K/UL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2887402,JC2474696,131277537399,2020-01-30 22:07:00+00:00,646392402,Inpatient,189,1,CANDIDA GLABRATA,0,None,...,"112.2,300.9,338.18,591,592.0,595.2,596.51,599....","B37.2,B37.41,D23.5,D27.1,D30.3,D62,D89.9,E03.9...","Metabolic Panel, Basic","CO2, Ser/Plas",20.00,2020-01-30 09:43:00+00:00,2020-01-30 10:08:00+00:00,2020-01-30 11:00:00+00:00,bicarb,MMOL/L
2887403,JC2474696,131277537399,2020-01-30 22:07:00+00:00,646392402,Inpatient,189,1,CANDIDA GLABRATA,0,None,...,"112.2,300.9,338.18,591,592.0,595.2,596.51,599....","B37.2,B37.41,D23.5,D27.1,D30.3,D62,D89.9,E03.9...",Glucose By Meter,Glucose by Meter,103.00,2020-01-31 14:41:00+00:00,2020-01-31 14:36:00+00:00,2020-01-31 14:36:00+00:00,gluc,G/DL
2887404,JC2474696,131277537399,2020-01-30 22:07:00+00:00,646392402,Inpatient,189,1,CANDIDA GLABRATA,0,None,...,"112.2,300.9,338.18,591,592.0,595.2,596.51,599....","B37.2,B37.41,D23.5,D27.1,D30.3,D62,D89.9,E03.9...",Glucose By Meter,Glucose by Meter,117.00,2020-01-31 06:47:00+00:00,2020-01-31 06:42:00+00:00,2020-01-31 06:42:00+00:00,gluc,G/DL
2887405,JC2474696,131277537399,2020-01-30 22:07:00+00:00,646392402,Inpatient,189,1,CANDIDA GLABRATA,0,None,...,"112.2,300.9,338.18,591,592.0,595.2,596.51,599....","B37.2,B37.41,D23.5,D27.1,D30.3,D62,D89.9,E03.9...","Metabolic Panel, Basic","BUN, Ser/Plas",20.00,2020-01-30 09:43:00+00:00,2020-01-30 10:08:00+00:00,2020-01-30 11:00:00+00:00,bun,G/DL


In [22]:
labs_df.to_gbq(
    destination_table='Fatemeh_db.uti_cohort_labs',
    project_id='som-nero-phi-jonc101',
    if_exists='replace'
)

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_48193/3619976142.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  labs_df.to_gbq(


## Vitals with 48 hours -- the most specific vitals that we need is temperature for labeling 

In [26]:
%%bigquery vitals_df
WITH cohort AS (
    SELECT * FROM som-nero-phi-jonc101.Fatemeh_db.uti_cohort
)
  SELECT c.anon_id,
  c.encounterid, 
   meas_value,
    numerical_val_1,
     numerical_val_2,
     numerical_val_3,
     numerical_val_4,
    units,
   recorded_time_jittered,
  CASE
  WHEN upper(row_disp_name) IN ('PULSE', 'HEART RATE') THEN 'hrate'
  WHEN upper(row_disp_name) in ('RESP', 'RESP RATE') THEN 'rrate'
  WHEN upper(row_disp_name) IN ('TEMP') THEN 'temp'
  WHEN upper(row_disp_name) in ('WEIGHT') THEN 'weight'
  ELSE row_disp_name
  END as vital_sign
  FROM
  `som-nero-phi-jonc101.shc_core_2023.flowsheet` as vitals INNER JOIN cohort c
   ON vitals.anon_id = c.anon_id
  WHERE
  ((upper(trim(row_disp_name)) IN ('PULSE', 'HEART RATE') AND SAFE_CAST(numerical_val_1 AS numeric) >= 30) OR -- Heart rate
   (upper(trim(row_disp_name)) in ('RESP', 'RESP RATE') AND SAFE_CAST(numerical_val_1 AS numeric) >= 4 AND SAFE_CAST(numerical_val_1 AS numeric) <= 60) OR -- Respiratory rate
   (upper(trim(row_disp_name)) IN ('TEMP') AND SAFE_CAST(numerical_val_1 AS numeric) >= 90) OR -- Temperature in F
   (upper(trim(row_disp_name)) IN ('WEIGHT') AND SAFE_CAST(numerical_val_1 AS numeric) >= 480 AND SAFE_CAST(numerical_val_1 AS numeric) <= 8000) --- OR -- Weight 
   ---(upper(trim(row_disp_name)) IN ('SPO2', 'RESTING SPO2') AND CAST(numerical_val_1 AS numeric) >= 70) OR -- SpO2
   --- (REGEXP_CONTAINS(UPPER(template), vs_group_name) AND REGEXP_CONTAINS(UPPER(row_disp_name), o2_delivery_regex) AND NOT REGEXP_CONTAINS(UPPER(row_disp_name), o2_delivery_regex_negate)) -- O2 Delivery (SpO2 is different)
  )
  AND
  (TIMESTAMP_DIFF(recorded_time_jittered_utc, c.order_time_jittered_utc, hour) between -24 and 24 )


/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/job/query.py:1892: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [27]:
vitals_df.drop_duplicates(inplace=True)
vitals_df

,anon_id,encounterid,meas_value,numerical_val_1,numerical_val_2,numerical_val_3,numerical_val_4,units,recorded_time_jittered,vital_sign
0,JC2759531,131260164331,28,28,None,None,None,None,2018-10-28 07:17:00,rrate
1,JC1201869,131085375080,34,34,None,None,None,None,2015-07-11 08:30:00,rrate
2,JC1289339,131257897278,100.9,100.9,None,None,None,None,2018-11-03 13:30:00,temp
11,JC1696472,131313913688,93.9,93.9,None,None,None,None,2021-08-10 22:15:00,temp
15,JC1273976,131198108584,102.9,102.9,None,None,None,None,2016-10-11 14:12:00,temp
...,...,...,...,...,...,...,...,...,...,...
6937598,JC1980761,131298890540,98.8,98.8,None,None,None,None,2020-12-27 07:38:00,temp
6937602,JC566315,131254858668,98.8,98.8,None,None,None,None,2018-08-23 01:29:00,temp
6937603,JC1408025,131277939462,98.8,98.8,None,None,None,None,2019-10-31 18:00:00,temp
6937604,JC2903910,131301995053,98.8,98.8,None,None,None,None,2021-01-23 16:15:00,temp


In [28]:
vitals_df.to_gbq(
    destination_table='Fatemeh_db.uti_cohort_vitals',
    project_id='som-nero-phi-jonc101',
    if_exists='replace'
)

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_48193/57904940.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  vitals_df.to_gbq(


## Add demographics 

In [29]:
%%bigquery demos_df
WITH cohort AS (
    SELECT * FROM som-nero-phi-jonc101.Fatemeh_db.uti_cohort
)
select c.anon_id,
c.encounterid,
DATE_DIFF(CAST(c.order_time_jittered_utc as DATE), demo.BIRTH_DATE_JITTERED, YEAR) as Age,
demo.gender,
demo.canonical_race as race,
demo.canonical_ethnicity as ethnic
 from cohort c left join `som-nero-phi-jonc101.shc_core_2023.demographic` demo using (anon_id) 

/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/job/query.py:1892: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [30]:
demos_df.drop_duplicates(inplace=True)
demos_df

,anon_id,encounterid,Age,gender,race,ethnic
0,JC940746,131084669273,42,Male,White,Non-Hispanic
4,JC1912907,131297076077,76,Female,Other,Hispanic/Latino
5,JC838860,131279784667,42,Female,Other,Hispanic/Latino
9,JC1687560,131224182591,66,Female,Asian,Non-Hispanic
58,JC527029,131200662657,77,Female,White,Non-Hispanic
...,...,...,...,...,...,...
73569,JC1013315,131283286450,57,Female,White,Non-Hispanic
73573,JC2840571,131274462171,31,Female,Asian,Non-Hispanic
73589,JC2561190,131265382843,55,Female,White,Non-Hispanic
73590,JC761986,131288738075,83,Male,Asian,Non-Hispanic


In [31]:
demos_df.to_gbq(
    destination_table='Fatemeh_db.uti_cohort_demos_df',
    project_id='som-nero-phi-jonc101',
    if_exists='replace'
)

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_48193/3888211553.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  demos_df.to_gbq(


## Lets Extract Statics from the Cohort

## patients population with posetive culture order 

## The end of scripts- the rests are test scripts

# Statistics Analysis 

In [ ]:
## ICD codes
icd9s_wide AS (
  SELECT distinct culture_labels.anon_id, culture_labels.order_proc_id_coded,
    STRING_AGG(distinct diagnoses.icd9) as icd9s
  FROM `som-nero-phi-jonc101.shc_core_2023.diagnosis` as diagnoses INNER JOIN
  culture_labels on culture_labels.anon_id = diagnoses.anon_id
  WHERE DATETIME_DIFF(culture_labels.order_time_jittered, diagnoses.start_date_jittered, DAY) > 0
  GROUP BY culture_labels.anon_id, culture_labels.order_proc_id_coded
),

In [ ]:
WITH cohort AS (
    SELECT * FROM som-nero-phi-jonc101.Fatemeh_db.uti_cohort
),
Antimicrobial_reciving as(
select op.*,
mm.RXCUI 
from `som-nero-phi-jonc101.shc_core_2023.order_med` op
left join (select * from `som-nero-phi-jonc101.shc_core_2023.mapped_meds` mm where RXCUI!='0') mm
on mm.name=op.med_description
where  op.thera_class_name in ('ANTIVIRALS','ANTIBIOTICS','ANTIINFECTIVES','ANTIFUNGALS','ANTIPARASITICS') 
)
select c.*,
1 as antimicrobail_recived,
ar.order_start_time_jittered_utc as antimicobial_start_time,
ar.order_end_time_jittered_utc as antimicobial_end_time,
ar.order_inst_jittered_utc as antimicobial_order_time,
from cohort c 
left join Antimicrobial_reciving ar 
using (anon_id) 
WHERE
TIMESTAMP_DIFF(ar.order_inst_jittered_utc,c.order_time_jittered_utc,HOUR) BETWEEN -24 AND 24 

In [2]:
%%bigquery df_uc_all
WITH urine_cultures AS ( -- all urine culture to be extracted 
    SELECT DISTINCT
        op.anon_id, 
        op.pat_enc_csn_id_coded as encounterid,
        op.order_proc_id_coded, 
        op.order_time_jittered_utc, 
        op.ordering_mode
    FROM 
        `som-nero-phi-jonc101.shc_core_2023.order_proc` op
     INNER JOIN
        `som-nero-phi-jonc101.shc_core_2023.lab_result` lr
    ON
        op.order_proc_id_coded = lr.order_id_coded
    WHERE
        op.order_type LIKE "Microbiology%"
        AND op.description LIKE "%URINE%"
        AND EXTRACT(YEAR FROM op.order_time_jittered_utc)>=2015 
        AND EXTRACT(YEAR FROM op.order_time_jittered_utc)<=2020
),
adult_urine_cultures AS (
    SELECT DISTINCT
        uc.*
    FROM 
        urine_cultures uc
    INNER JOIN
        `som-nero-phi-jonc101.shc_core_2023.demographic` demo
    USING
        (anon_id)
    WHERE
        DATE_DIFF(CAST(uc.order_time_jittered_utc as DATE), demo.BIRTH_DATE_JITTERED, YEAR) >= 18
),


-- Must not have any urine culture orders (that go on to result in lab results) in prior two weeks
-- This finds urine cultures that do have other urine cultures orders in prior two weeks
order_in_prior_two_weeks AS (
    SELECT DISTINCT
         auc.order_proc_id_coded
    FROM 
        `mining-clinical-decisions.shc_core.order_proc` op
     INNER JOIN
        `som-nero-phi-jonc101.shc_core_2023.lab_result` lr
    ON
        op.order_proc_id_coded = lr.order_id_coded
    INNER JOIN
        adult_urine_cultures auc 
    ON
        op.anon_id = auc.anon_id
    WHERE
        op.order_type LIKE "Microbiology%"
        AND op.description LIKE "%URINE%"
        AND auc.order_time_jittered_utc > op.order_time_jittered_utc
        AND TIMESTAMP_DIFF(auc.order_time_jittered_utc, op.order_time_jittered_utc, DAY) < 14
),

included_adult_urine_cultures AS (
    SELECT DISTINCT
        auc.*
    FROM 
        adult_urine_cultures auc
    WHERE 
        auc.order_proc_id_coded NOT IN (SELECT order_proc_id_coded  FROM order_in_prior_two_weeks)
),
-- finf prior uti order up to 1 year before 
prior_utis AS (
    SELECT 
        auc.*,
        op.order_proc_id_coded as PriorUTI_proc_code,
        TIMESTAMP_DIFF(auc.order_time_jittered_utc, op.order_time_jittered_utc, Day) as TSLU,
    
        ROW_NUMBER() OVER (
            PARTITION BY op.anon_id
            ORDER BY ABS(TIMESTAMP_DIFF(auc.order_time_jittered_utc, op.order_time_jittered_utc, SECOND))
        ) AS rn
    FROM 
        `som-nero-phi-jonc101.shc_core_2023.order_proc` op
    INNER JOIN
        `som-nero-phi-jonc101.shc_core_2023.lab_result` lr
    ON
        op.order_proc_id_coded = lr.order_id_coded
    Right JOIN
        included_adult_urine_cultures auc 
    ON
        op.anon_id = auc.anon_id
    WHERE
        op.order_type LIKE "Microbiology%"
        AND op.description LIKE "%URINE%"
        AND auc.order_time_jittered_utc > op.order_time_jittered_utc
        AND TIMESTAMP_DIFF(auc.order_time_jittered_utc, op.order_time_jittered_utc, DAY) < 365
),

included_adult_urine_cultures_with_TSLU as (
select *
from prior_utis 
where rn=1
),

Include_abx_list1 AS (
    SELECT
        med_description
    FROM
        `som-nero-phi-jonc101.abx.abx_types`  
    WHERE
        is_include_abx = 1 OR is_oral = 1
),
Include_abx as (
select med_description 
WHERE REGEXP_CONTAINS(UPPER(med_description),
r'cephalexin|keflex|amoxicillin|ciprofloxacin|levofloxacin|nitrofurantoin|trimethoprim|bactrim|septra|macrobid|macrodantin|levaquin|cipro|augmentation')),

ordered_abx AS (
    SELECT 
        auc.*,
        om.med_description 
    FROM 
        adult_urine_cultures auc
    INNER JOIN
        `som-nero-phi-jonc101.shc_core_2023.order_med` om
    USING
        (anon_id)
    WHERE
        om.med_description IN (SELECT med_description FROM include_abx)
    AND 
        TIMESTAMP_DIFF(om.order_inst_jittered_utc, auc.order_time_jittered_utc, day) BETWEEN -28 AND 0 
),
included_urine_cultures AS (
    SELECT DISTINCT
        auc.*, oa.med_description
    FROM 
        adult_urine_cultures auc
    LEFT JOIN
        ordered_abx oa
    USING
        (order_proc_id_coded)
    WHERE 
        auc.order_proc_id_coded NOT IN (SELECT order_proc_id_coded  FROM order_in_prior_two_weeks)
),

-- Flag to indicated if the culture was positive, and resulting organisms
orders_that_were_positive AS (
    SELECT DISTINCT
        auc.order_proc_id_coded, 1 as was_positive, organism
    FROM
        included_urine_cultures auc
    INNER JOIN  
        (SELECT DISTINCT order_proc_id_coded, organism
        FROM `som-nero-phi-jonc101.shc_core_2023.culture_sensitivity`) cs
    USING
        (order_proc_id_coded) 
),
-- check other posetive culture within7 days of urin culture (e.g useful for silver labeling)
orders_with_other_pos_cultures AS (
    SELECT DISTINCT
        auc.order_proc_id_coded,
        1 as had_other_pos_culture,
        description other_pos_site,
        organism other_organism
    FROM
        included_urine_cultures auc
    INNER JOIN  
        `som-nero-phi-jonc101.shc_core_2023.culture_sensitivity` cs
    USING
        (anon_id) 
    WHERE
        auc.order_proc_id_coded <> cs.order_proc_id_coded
    AND
        -- Some urine culture orders have two order ids where only one results, make sure we're not counting these.
        -- Likely not needed logic after forcing orders to appear in lab_results
        (auc.order_time_jittered_utc <> cs.order_time_jittered_utc OR cs.description NOT LIKE '%URINE%')
    AND
        TIMESTAMP_DIFF(cs.order_time_jittered_utc, auc.order_time_jittered_utc, Day)
    BETWEEN
        0 AND 7
),


--Temporary building block that has medications and organisms in long form
orders_meds_and_bugs AS (
SELECT DISTINCT
    anon_id, 
    encounterid, 
    order_proc_id_coded, 
    order_time_jittered_utc,
    ordering_mode, 
    CASE WHEN was_positive = 1 THEN 1 ELSE 0 END was_positive,
    CASE WHEN had_other_pos_culture = 1 THEN 1 ELSE 0 END had_other_pos_culture,
    med_description,
    organism,
    other_pos_site,
    other_organism
FROM 
    included_urine_cultures
LEFT JOIN 
    orders_with_other_pos_cultures
USING 
    (order_proc_id_coded)
LEFT JOIN 
    orders_that_were_positive
USING
    (order_proc_id_coded)
ORDER BY
    order_proc_id_coded
),

-- Collapse medication orders to order id of the urine culture
concatenated_meds AS (
    SELECT DISTINCT
        order_proc_id_coded,
        STRING_AGG(DISTINCT med_description ORDER BY med_description) ordered_antibiotics
    FROM
        orders_meds_and_bugs
    GROUP BY
        order_proc_id_coded
),
-- Collpase organisms to order id of the urine culture
concatenated_bugs AS (
    SELECT DISTINCT
        order_proc_id_coded,
        STRING_AGG(DISTINCT organism ORDER BY organism) organisms
    FROM
        orders_meds_and_bugs
    GROUP BY
        order_proc_id_coded
),
-- Collpase other positive sites
concatenated_sites AS (
    SELECT DISTINCT
        order_proc_id_coded,
        STRING_AGG(DISTINCT other_pos_site ORDER BY other_pos_site) other_pos_sites
    FROM
        orders_meds_and_bugs
    GROUP BY
        order_proc_id_coded
),
concatenated_other_bugs AS (
    SELECT DISTINCT
        order_proc_id_coded,
        STRING_AGG(DISTINCT other_organism ORDER BY other_organism) other_organisms
    FROM
        orders_meds_and_bugs
    GROUP BY
        order_proc_id_coded
)
SELECT DISTINCT
    anon_id,
    encounterid,
    order_proc_id_coded, 
    order_time_jittered_utc, 
    ordering_mode,
     was_positive,
      had_other_pos_culture,
    ordered_antibiotics,
     organisms,
      other_pos_sites,
       other_organisms
FROM
    orders_meds_and_bugs
INNER JOIN 
    concatenated_meds
USING
    (order_proc_id_coded)
INNER JOIN 
    concatenated_bugs 
USING
    (order_proc_id_coded)
INNER JOIN 
    concatenated_sites  
USING
    (order_proc_id_coded)
INNER JOIN 
    concatenated_other_bugs  
USING
    (order_proc_id_coded)
ORDER BY 
    order_proc_id_coded;


/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/job/query.py:1892: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [3]:
print(df_uc_all.shape)
df_uc_all.head(5)

(423185, 11)


,anon_id,encounterid,order_proc_id_coded,order_time_jittered_utc,ordering_mode,was_positive,had_other_pos_culture,ordered_antibiotics,organisms,other_pos_sites,other_organisms
0,JC638777,6619001,300630080,2003-02-05 20:13:00+00:00,Outpatient,0,0,None,None,None,None
1,JC511881,14533332,300631891,2000-08-13 16:33:00+00:00,Outpatient,0,0,None,None,None,None
2,JC512088,6619302,300635318,2001-09-20 18:30:00+00:00,Outpatient,0,0,None,None,None,None
3,JC640175,6619573,300640720,2003-12-03 17:46:00+00:00,Outpatient,0,0,None,None,None,None
4,JC512077,6619661,300642470,2000-08-03 23:20:00+00:00,Outpatient,0,0,None,None,None,None


In [4]:
%%bigquery df_prior_abx
-- Assuming above query has been saved to mining-clinical-decisions.conor_db.uti_cohort
WITH cohort AS (
    SELECT * FROM som-nero-phi-jonc101.Fatemeh_db.uti_cohort
),

-- Set of antibiotics that show up in order_med (may not be totally all encompassing) 
include_abx AS (
    SELECT
        med_description
    FROM
        `som-nero-phi-jonc101.abx.abx_types` 
    WHERE
        is_include_abx = 1 OR is_oral = 1
),

prior_abx AS (
    SELECT DISTINCT
        cohort.order_proc_id_coded
    FROM 
        cohort
    INNER JOIN
        `som-nero-phi-jonc101.shc_core_2023.order_med` om
    USING
        (anon_id)
    WHERE
        om.med_description IN (SELECT med_description FROM include_abx)
    AND 
        TIMESTAMP_DIFF(om.order_inst_jittered_utc, cohort.order_time_jittered_utc, HOUR) BETWEEN -24*28 AND 0 
)

-- Remove observations with prior antibiotics
SELECT DISTINCT
    *
FROM 
    cohort
WHERE 
    order_proc_id_coded NOT IN (SELECT DISTINCT order_proc_id_coded FROM prior_abx)


/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/job/query.py:1892: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/Users/fa/Documents/GCP_env/env_gcp/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [5]:
print(df_prior_abx.shape)
df_prior_abx.head()

(267856, 11)


,anon_id,encounterid,order_proc_id_coded,order_time_jittered_utc,ordering_mode,was_positive,had_other_pos_culture,ordered_antibiotics,organisms,other_pos_sites,other_organisms
0,JC727692,6620480,300657470,2003-03-02 23:45:00+00:00,Outpatient,0,0,None,None,None,None
1,JC643163,6620898,300665289,2000-12-10 22:01:00+00:00,Outpatient,0,0,None,None,None,None
2,JC515072,6621267,300671890,2003-02-19 16:00:00+00:00,Outpatient,0,0,None,None,None,None
3,JC734372,6624702,300734017,2003-09-02 22:12:00+00:00,Outpatient,0,0,None,None,None,None
4,JC736826,6626467,300764811,2000-07-16 16:59:00+00:00,Outpatient,0,0,None,None,None,None


In [6]:
def string_agg_cols(arr):
    final_set = []
    for a in arr:
        if a is None:
            continue
        groups = a.split(',')
        for g in groups:
            if g not in final_set:
                final_set.append(g)
    final_set.sort()
    if len(final_set) == 0:
        return None
    else:
        return ','.join([f for f in final_set])
    
groupby_cols = ['anon_id', 'encounterid', 'order_time_jittered_utc']
df_final = (df_prior_abx
    .groupby(groupby_cols)
    .agg({'order_proc_id_coded' : 'first',
          'ordering_mode' : 'first',
          'was_positive' : 'max',
          'had_other_pos_culture' : 'max',
          'ordered_antibiotics' : string_agg_cols,
          'organisms' : string_agg_cols,
          'other_pos_sites' : string_agg_cols,
          'other_organisms' : string_agg_cols})
    .reset_index()
)
print(df_final.shape)
df_final.head()

(264487, 11)


,anon_id,encounterid,order_time_jittered_utc,order_proc_id_coded,ordering_mode,was_positive,had_other_pos_culture,ordered_antibiotics,organisms,other_pos_sites,other_organisms
0,JC1000010,15404221,2008-05-12 03:14:00+00:00,325204620,Outpatient,1,0,None,ESCHERICHIA COLI,None,None
1,JC1000013,15174722,2008-04-11 23:45:00+00:00,302914865,Outpatient,1,0,None,ESCHERICHIA COLI,None,None
2,JC1000021,32992898,2009-06-06 23:12:00+00:00,351382857,Outpatient,0,0,None,None,None,None
3,JC1000021,131009159044,2010-05-29 23:22:00+00:00,366795029,Outpatient,0,0,None,None,None,None
4,JC1000022,15142402,2008-03-13 03:50:00+00:00,323829550,Outpatient,1,0,None,ESCHERICHIA COLI,None,None


In [9]:
df_final.to_gbq(
    destination_table='Fatemeh_db.uti_cohort_final',
    project_id='som-nero-phi-jonc101',
    if_exists='replace'
)

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_38814/651619523.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_final.to_gbq(


In [10]:
df_uc_all = df_final

In [12]:
df_uc_all = df_uc_all[~df_uc_all.duplicated(subset=['anon_id', 'encounterid', 'order_time_jittered_utc'])]
df_uc_all.shape

(264487, 11)

In [14]:
df_uc_all.ordering_mode.value_counts(dropna=False)

ordering_mode
Outpatient    166881
Inpatient      92910
None            4696
Name: count, dtype: int64

In [13]:
df_uc_all.was_positive.value_counts()

was_positive
0    212701
1     51786
Name: count, dtype: Int64

In [15]:
print("Outpatient positivity rate")
print(df_uc_all.query('ordering_mode == "Outpatient"', engine='python')['was_positive'].mean())
print("Inpatient positivity rate")
print(df_uc_all.query('ordering_mode == "Inpatient"', engine='python')['was_positive'].mean())

Outpatient positivity rate
0.15189865832539354
Inpatient positivity rate
0.2657087504036164


In [16]:
df_uc_all.had_other_pos_culture.value_counts()

had_other_pos_culture
0    261199
1      3288
Name: count, dtype: Int64

In [17]:
df_uc_all[['was_positive', 'had_other_pos_culture']].value_counts()

was_positive  had_other_pos_culture
0             0                        210784
1             0                         50415
0             1                          1917
1             1                          1371
Name: count, dtype: int64

## Of all cases where urine culture was negative but another culture was positive, what were most common anatomical sites of the positive cultures?


In [18]:
(df_uc_all
.query('was_positive == 0 and had_other_pos_culture == 1', engine='python')
['other_pos_sites']
).value_counts()

other_pos_sites
URINE CULTURE                                                                       352
BLOOD CULTURE (AEROBIC & ANAEROBIC BOTTLES)                                         247
BLOOD CULTURE (2 AEROBIC BOTTLES)                                                   194
BLOOD CULTURE (AEROBIC & ANAEROBIC BOTTLE)                                          185
WOUND CULTURE                                                                       125
                                                                                   ... 
RESPIRATORY CULTURE AND GRAM STAIN,URINE CULTURE                                      1
BIOPSY/TISSUE WITH GRAM STAIN,WOUND CULTURE                                           1
ORTHOPEDIC AEROBIC CULT W/ GRAM STAIN                                                 1
 DEEP / SURGICAL / ASP,FLUID CULTURE AND GRAM STAIN,WOUND CULTURE AND GRAM STAIN      1
BLOOD CULTURE (2 AEROBIC BOTTLES),FUNGAL CULTURE                                      1
Name: count, Len

## Most common antibiotic orders

In [19]:
df_uc_all.ordered_antibiotics.value_counts(dropna=False).head(20)

ordered_antibiotics
None                                                                      213411
NITROFURANTOIN MONOHYD/M-CRYST 100 MG PO CAPS                               6493
CEPHALEXIN 500 MG PO CAPS                                                   3429
CIPROFLOXACIN HCL 500 MG PO TABS                                            2339
SULFAMETHOXAZOLE-TRIMETHOPRIM 800-160 MG PO TABS                            2276
CIPROFLOXACIN 500 MG PO TABS                                                1416
AMOXICILLIN-POT CLAVULANATE 875-125 MG PO TABS                               806
CEFTRIAXONE 1 GRAM/10 ML IV PUSH (VIAL ONLY)                                 743
CEFAZOLIN 1 GRAM INJ SOLR                                                    717
CIPROFLOXACIN HCL 250 MG PO TABS                                             672
MACROBID 100 MG PO CAPS                                                      484
CEFTRIAXONE 1 GRAM/10 ML IV PUSH (VIAL ONLY),CEPHALEXIN 500 MG PO CAPS       477
LEVOFLOX

## Most common arganism

In [20]:
df_uc_all.organisms.value_counts(dropna=False).head(20)

organisms
None                                      212701
ESCHERICHIA COLI                           28172
KLEBSIELLA PNEUMONIAE                       4575
ENTEROCOCCUS SPECIES                        4049
PROTEUS MIRABILIS                           1756
STREPTOCOCCUS AGALACTIAE (GROUP B)          1684
ENTEROCOCCUS SPECIES,ESCHERICHIA COLI       1240
STREPTOCOCCUS AGALACTIAE {GROUP B}           976
PSEUDOMONAS AERUGINOSA                       908
ESCHERICHIA COLI,KLEBSIELLA PNEUMONIAE       580
KLEBSIELLA OXYTOCA                           580
ENTEROCOCCUS FAECALIS                        495
ENTEROBACTER CLOACAE COMPLEX                 476
CITROBACTER KOSERI                           434
CITROBACTER FREUNDII COMPLEX                 319
KLEBSIELLA AEROGENES                         286
ESCHERICHIA COLI,PROTEUS MIRABILIS           281
STAPHYLOCOCCUS AUREUS                        279
ZZZENTEROBACTER AEROGENES                    257
MORGANELLA MORGANII                          190
Name: coun

In [21]:
df_uc_all.assign(year=lambda x: x.order_time_jittered_utc.dt.year)[['year']].value_counts()

year
2023    27673
2022    24344
2018    21918
2021    21697
2019    21487
2017    19620
2020    19226
2016    18134
2015    13193
2013     9561
2009     9298
2008     9250
2014     9026
2011     8988
2012     8883
2010     8811
2001     2232
2000     2231
2002     1826
2003     1722
2004     1666
2005     1581
2006     1485
2024      568
1999       66
2007        1
Name: count, dtype: int64

In [22]:
df_uc_all.head()


,anon_id,encounterid,order_time_jittered_utc,order_proc_id_coded,ordering_mode,was_positive,had_other_pos_culture,ordered_antibiotics,organisms,other_pos_sites,other_organisms
0,JC1000010,15404221,2008-05-12 03:14:00+00:00,325204620,Outpatient,1,0,None,ESCHERICHIA COLI,None,None
1,JC1000013,15174722,2008-04-11 23:45:00+00:00,302914865,Outpatient,1,0,None,ESCHERICHIA COLI,None,None
2,JC1000021,32992898,2009-06-06 23:12:00+00:00,351382857,Outpatient,0,0,None,None,None,None
3,JC1000021,131009159044,2010-05-29 23:22:00+00:00,366795029,Outpatient,0,0,None,None,None,None
4,JC1000022,15142402,2008-03-13 03:50:00+00:00,323829550,Outpatient,1,0,None,ESCHERICHIA COLI,None,None


In [23]:
df_uc_all[df_uc_all.ordered_antibiotics.notna()]

,anon_id,encounterid,order_time_jittered_utc,order_proc_id_coded,ordering_mode,was_positive,had_other_pos_culture,ordered_antibiotics,organisms,other_pos_sites,other_organisms
7,JC1000063,131011127694,2010-09-30 16:47:00+00:00,373759075,Inpatient,0,0,"DOXYCYCLINE HYCLATE 100 MG PO CAPS,DOXYCYCLINE...",None,None,None
31,JC1000196,131012857817,2011-03-09 02:52:00+00:00,381149512,Outpatient,0,0,CEFAZOLIN IN DEXTROSE (ISO-OS) 1 GRAM/50 ML IV...,None,None,None
44,JC1000441,131074471154,2015-02-26 15:27:00+00:00,458813098,Inpatient,0,0,CEFAZOLIN 1 GRAM INJ SOLR,None,None,None
54,JC1000583,131214619922,2017-01-31 02:46:00+00:00,515522887,Outpatient,1,0,"CIPROFLOXACIN HCL 500 MG PO TABS,SULFAMETHOXAZ...",ENTEROCOCCUS SPECIES,None,None
57,JC1000610,131094117826,2015-06-20 21:54:00+00:00,469086862,Inpatient,0,0,"CEFTRIAXONE 1 GRAM/50 ML MINI-BAG PLUS,CEPHALE...",None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
264469,JC999695,131238748562,2017-08-31 23:50:00+00:00,536694632,Inpatient,0,0,MOXIFLOXACIN 400 MG PO TABS,None,None,None
264470,JC999695,131240032823,2017-10-05 20:26:00+00:00,539891931,Inpatient,0,1,ERTAPENEM IVPB (CUSTOM),None,BLOOD CULTURE (2 AEROBIC BOTTLES),ENTEROBACTER CLOACAE COMPLEX
264478,JC999841,131042520339,2014-10-10 04:28:00+00:00,447102934,Inpatient,0,0,NITROFURANTOIN MONOHYD/M-CRYST 100 MG PO CAPS,None,None,None
264483,JC999925,131107270857,2015-08-09 09:12:00+00:00,471989931,Inpatient,0,0,SULFAMETHOXAZOLE-TRIMETHOPRIM 400-80 MG PO TABS,None,None,None


In [40]:
df_uc_all.to_gbq(
    destination_table='Fatemeh_db.uti_cohort_final',
    project_id='som-nero-phi-jonc101',
    if_exists='replace'
)

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_38814/2162197290.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_uc_all.to_gbq(


## Labs within last 14 days

In [36]:
# Assuming 'df' is your dataframe
#selected_rows = df_wbc[df_wbc['encounterid'] != df_wbc['labsenc']]

In [47]:
labs_results

,anon_id,encounterid,order_time_jittered_utc,order_proc_id_coded,ordering_mode,was_positive,had_other_pos_culture,ordered_antibiotics,organisms,other_pos_sites,other_organisms,ord_value,ord_num_value,result_time_jittered,lab,unit,rn
0,JC1000013,15174722,2008-04-11 23:45:00+00:00,302914865,Outpatient,1,0,None,ESCHERICHIA COLI,None,None,12.0,12.000000000,2008-04-14 08:27:00,hgb,G/DL,1
1,JC1000013,15174722,2008-04-11 23:45:00+00:00,302914865,Outpatient,1,0,None,ESCHERICHIA COLI,None,None,248,248.000000000,2008-04-14 08:27:00,plt,K/UL,1
2,JC1000013,15174722,2008-04-11 23:45:00+00:00,302914865,Outpatient,1,0,None,ESCHERICHIA COLI,None,None,14.5,14.500000000,2008-04-14 08:27:00,wbc,K/UL,1
3,JC1000013,15174722,2008-04-11 23:45:00+00:00,302914865,Outpatient,1,0,None,ESCHERICHIA COLI,None,None,142,142.000000000,2008-04-14 08:58:00,sodium,MMOL/L,1
4,JC1000013,15174722,2008-04-11 23:45:00+00:00,302914865,Outpatient,1,0,None,ESCHERICHIA COLI,None,None,8,8.000000000,2008-04-14 08:58:00,bun,G/DL,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47517961,JC999992,131189547541,2016-06-06 03:57:00+00:00,497201103,Outpatient,0,0,CEPHALEXIN 500 MG PO CAPS,None,None,None,0.87,0.870000000,2016-06-11 20:15:00,creat,G/DL,1
47517962,JC999992,131189547541,2016-06-06 03:57:00+00:00,497201103,Outpatient,0,0,CEPHALEXIN 500 MG PO CAPS,None,None,None,101,101.000000000,2016-06-11 20:15:00,gluc,G/DL,1
47517963,JC999998,15725437,2008-06-14 04:25:00+00:00,326716282,Outpatient,1,0,None,ESCHERICHIA COLI,None,None,14.5,14.500000000,2006-04-26 19:15:00,hgb,G/DL,1
47517964,JC999998,15725437,2008-06-14 04:25:00+00:00,326716282,Outpatient,1,0,None,ESCHERICHIA COLI,None,None,12.3,12.300000000,2006-04-26 19:15:00,wbc,K/UL,1


In [ ]:
## Medications 